In [ ]:
# !pip install tabula-py
!pip install pdfplumber

You should consider upgrading via the 'C:\Users\Admin\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [2]:
import tabula

In [ ]:
import pandas as pd
import pdfplumber

# File paths
file_path_p4 = r"ueh_graduating_score_overview_Dot4_2024.pdf"
file_path_p3 = r"ueh_graduating_score_overview_Dot3_2024.pdf"

# Function to extract tables from a PDF using pdfplumber
def extract_tables_with_plumber(pdf_path):
    extracted_tables = []  # Khởi tạo danh sách để lưu các bảng trích xuất từ PDF
    with pdfplumber.open(pdf_path) as pdf:  # Mở tệp PDF bằng pdfplumber
        for page_num, page in enumerate(pdf.pages, start=1):  # Lặp qua từng trang
            tables = page.extract_tables() # Trích xuất tất cả bảng từ trang hiện tại
            for table in tables:  # Duyệt qua từng bảng trong danh sách các bảng
                # Convert the table into a DataFrame
                df = pd.DataFrame(table[1:], columns=table[0])  # Chuyển bảng thành DataFrame
                extracted_tables.append(df) # Thêm DataFrame vào danh sách
    return extracted_tables

# Extract tables from both PDF files
dfs_p4 = extract_tables_with_plumber(file_path_p4)
dfs_p3 = extract_tables_with_plumber(file_path_p3)

# Add "Đợt xét tốt nghiệp" column to each DataFrame
for df in dfs_p4:
    df['Đợt xét tốt nghiệp'] = '20241201'

for df in dfs_p3:
    df['Đợt xét tốt nghiệp'] = '20240901'

# Combine all DataFrames into a single DataFrame
combined_df = pd.concat(dfs_p4 + dfs_p3, ignore_index=True)


In [4]:
# Bước 1: Chuyển đổi xếp loại thành một giá trị số để dễ dàng sắp xếp
# Chuyển đổi thứ tự ưu tiên: Xuất sắc > Giỏi > Khá > Trung bình khá
rank_map = {
    'Xuất sắc': 1,
    'Giỏi': 2,
    'Khá': 3,
    'Trung bình khá': 4
}

combined_df['Xếp loại số'] = combined_df['Xếp loại'].map(rank_map)

# Bước 2: Sắp xếp theo ĐTB TN (giảm dần) và Xếp loại số (tăng dần)
combined_df = combined_df.sort_values(by=['ĐTB TN', 'Xếp loại số'], ascending=[False, True])

# Bước 3: Tạo cột Thứ hạng toàn khóa và Thứ hạng trong chuyên ngành
combined_df['Thứ hạng toàn khóa'] = combined_df.groupby('Khóa học')['ĐTB TN'].rank(method='dense', ascending=False)
combined_df['Thứ hạng trong chuyên ngành'] = combined_df.groupby(['Khóa học', 'Ngành', 'Chuyên ngành'])['ĐTB TN'].rank(method='dense', ascending=False)

In [5]:
combined_df = combined_df.drop(columns=['Xếp loại số'])

In [6]:
combined_df.to_csv("ueh_graduating_score_overview.csv", index=False, encoding='utf-8-sig')